In [100]:
from index import *
from weighter import *
from irmodel import *

In [277]:
from sklearn.metrics import mean_squared_error as mse
import copy
from numpy.random import sample
class RandomWalk:
    def __init__(self):
        return self
    def randomwalk(self):
        raise NotImplementedError('Always abstract class')       
class PageRank(RandomWalk):
    def __init__(self,max_iter,d):
        self.max_iter=max_iter
        self.d=d
    def randomwalk(self,linkout,linkin):
        self.mu={}
#         inital value
# linkout sortit de point i
# linkin enter in point i
# (V,E)=(all the document, link enter in this page =link in)

        N=len(linkout)*1.
        for page in linkout:
            self.mu[page]=1./N
        for t in range(self.max_iter):   
            mu_last=copy.deepcopy(self.mu)
            
            for i in linkout:
                Amu_i=0
                if linkin.has_key(i):
                    for j in linkin[i]:
                        Amu_i+=1.*self.mu[j]/len(linkout[j])# matrix compute
                self.mu[i]=1.*(1.-self.d)/N+self.d*Amu_i  
             
            print mse(self.mu.values(),mu_last.values())*len(mu_last.values())
        return self.mu  
    
class HITS(RandomWalk):
    def __init__(self,max_iter):
        self.max_iter=max_iter
    def randomwalk(self,linkout,linkin):
        self.a={}
        self.h={}
#         initial values
        for node in linkout:
            self.a[node]=1.
            self.h[node]=1.
        for t in range(self.max_iter):
            a_last=copy.deepcopy(self.a)
#             print a_last['1']
            h_last=copy.deepcopy(self.h)
            for i in linkout:         
                if linkin.has_key(i):
                    self.a[i]=np.sum(self.h[j] for j in list(linkin[i]))*1.
                    self.h[i]=np.sum(self.a[j] for j in list(linkout[i]))*1.   
                else:
                    self.a[i]=0.
                    self.h[i]=np.sum(self.a[j] for j in list(linkout[i])) *1.   
            a_norm=1.*np.linalg.norm(self.a.values(),ord=2)
            h_norm=1.*np.linalg.norm(self.h.values(),ord=2)
#             print self.a['593']
            for k in self.a:
                self.a[k]=1.*self.a[k]/a_norm
            for k in self.h:
                self.h[k]=1.*self.h[k]/h_norm
#             print float(self.a['593'])
#             print 4204*mse(self.a.values(),a_last.values())+4204*mse(self.h.values(),h_last.values())
        return self.a
from irmodel import *
import random
class ModelRandomWork(IRmodel):
    def __init__(self,irmodel,index,weighter,randomwalk,nb_seeds,k):
        
#         random walk , model, index are instance
        self.weighter=weighter
        self.irmodel=irmodel
        self.nb_seeds=nb_seeds
        self.randomwalk=randomwalk
        self.k=k
        self.index=index   
        IRmodel.__init__(self,weighter)
    def getScores(self,query):
        print self.index
        docs=[]
#         three kinds of model: vector language okapi
        ranking=self.irmodel.getRanking(query)
    
        docs=[k for k,value in ranking]
#         inition VQ
        seeds=docs[:self.nb_seeds]
        V_Q=set() # unique value 
        V_Q=set(seeds)
        for i in range(self.nb_seeds):
            for j in self.weighter.index.linkout[seeds[i]]:
                V_Q.add(j)
            if self.weighter.index.linkin.has_key(seeds[i]) and (len(self.weighter.index.linkin[seeds[i]])>=k):
                V_Q |=set(random.sample(list(self.weighter.index.linkin[seeds[i]]),self.k if len(self.weighter.index.linkin[seeds[i]]) > self.k else len(self.weighter.index.linkin[seeds[i]])))

#       sous graphe (VQ,EQ)

        souslinkouts = {}
        souslinkins = {}
        for doc in V_Q:
            souslinkouts[doc] = set()
            if doc not in souslinkins:
                souslinkins[doc] = set()
            for s in self.weighter.index.linkout[doc]:
                if s in V_Q:
                    souslinkouts[doc].add(s)
                    if s not in souslinkins:
                        souslinkins[s] = set()
                    souslinkins[s].add(doc)
# random walk  is a instance already defined  
#         if self.randomWalk == 1:
#             rw = PageRank(0.85, 20)
#         else:
#             rw = HITS(20)
        self.scores = self.randomwalk.randomwalk(souslinkouts, souslinkins) 
        return self.scores


In [258]:
a=set()
b=set()
b.add(1)
a |=b
print a

set([1])


In [200]:
w=WeighterTfTf()
w.construct()

In [202]:
w1=WeighterTfTf()

In [215]:
# v=Vectoriel(weighter=WeighterTfTf())
# v.construct()
# v.WeightsNormfordocs()

In [274]:
len(i.linkin)

3137

In [281]:
gq.queries['58'].relevants

['432',
 '536',
 '1293',
 '1344',
 '1398',
 '1411',
 '1420',
 '1445',
 '1619',
 '1629',
 '1631',
 '1691',
 '1709',
 '1812',
 '1944',
 '2098',
 '2115',
 '2122',
 '2123',
 '2249',
 '2349',
 '2395',
 '2634',
 '2636',
 '2719',
 '2731',
 '2825',
 '3159',
 '3166',
 '3167']

In [283]:
mrw=ModelRandomWork(irmodel=v,index=w.index,weighter=w1,randomwalk=prank,nb_seeds=1000,k=10)
r=mrw.getRanking(gq.queries['58'].text)

<index.Index instance at 0x111549ea8>
inition time is: 1481143079.31
0.0247762203217
0.00014186546154
0.000193344027742
0.000210695971465
0.000218081947897
0.000222039047649
0.000224221588595
0.000225468241659
0.000226201454115
0.00022664340151
0.000226915066773


In [285]:
len(r)

3652

In [246]:
import random
a=[1,2]
random.sample(a,2)

[2, 1]

In [270]:
hit=HITS(200)


In [102]:
from weighter import *
from index import *
w=WeighterTfTf()
v=Vectoriel(weighter=w)
v.construct()
v.WeightsNormfordocs()


<irmodel.Vectoriel instance at 0x1131260e0>

In [160]:
prank=PageRank(10,0.85)

In [288]:
gq.queries['58'].relevants

['432',
 '536',
 '1293',
 '1344',
 '1398',
 '1411',
 '1420',
 '1445',
 '1619',
 '1629',
 '1631',
 '1691',
 '1709',
 '1812',
 '1944',
 '2098',
 '2115',
 '2122',
 '2123',
 '2249',
 '2349',
 '2395',
 '2634',
 '2636',
 '2719',
 '2731',
 '2825',
 '3159',
 '3166',
 '3167']

In [287]:
mrw=ModelRandomWork(irmodel=v,index=w.index,weighter=w,randomwalk=hit,nb_seeds=4000,k=10)## Random walk mesure is 
mrw.getRanking(gq.queries['58'].text)

<index.Index instance at 0x10fa41560>
inition time is: 1481143200.41
0.024078130722


[('1366', 0.76556628003346427),
 ('1565', 0.39514521673130165),
 ('1188', 0.33286448427734672),
 ('1788', 0.18372016239114516),
 ('560', 0.16643223612973476),
 ('795', 0.14339207426102937),
 ('474', 0.14330682663575803),
 ('1781', 0.14330682659063393),
 ('720', 0.1433068146684168),
 ('2929', 0.033515411908297668),
 ('1215', 0.022611758569817217),
 ('984', 0.021883243420405818),
 ('989', 0.021872223983181487),
 ('1989', 0.017044342583021246),
 ('1364', 0.008923161711527558),
 ('288', 0.0089228383166284619),
 ('2921', 0.0080106863376222942),
 ('2060', 0.0080106294584820457),
 ('1988', 0.0058217407900404477),
 ('2252', 0.0058216264125444369),
 ('987', 0.004804064068705061),
 ('3188', 0.0044393748427255025),
 ('1184', 0.003213812627543465),
 ('1389', 0.0029807357920046424),
 ('2325', 0.0029107907266516658),
 ('1792', 0.0024061042941122854),
 ('267', 0.0024020269436918224),
 ('988', 0.0023999885052745306),
 ('2192', 0.0023984499340250105),
 ('3184', 0.0023979514814999286),
 ('2256', 0.00239

In [175]:
i=Index()
i.indexation()

<index.Index instance at 0x11e60e170>

In [179]:
i.getTfsForDoc('3699')

{'nondegen': 1,
 'multiplexor': 2,
 'prefix': 1,
 'lori': 2,
 'buddi': 1,
 '473l': 1,
 'nonmathemat': 1,
 'honeywel': 2,
 'govern': 1,
 'affect': 2,
 'vast': 1,
 'scholar': 1,
 'vanhorn': 1,
 'wegner': 1,
 '2314': 2,
 'vector': 1,
 'markov': 1,
 'esop': 1,
 'rodman': 1,
 'bacon': 1,
 'sturg': 4,
 'budget': 1,
 '272': 1,
 '275': 1,
 'illumin': 2,
 'outchannel': 1,
 'pdp': 1,
 'carr': 1,
 'net': 1,
 'hellberg': 1,
 'elimin': 1,
 'wasington': 1,
 'termedi': 1,
 'macroscop': 1,
 '108': 1,
 'china': 1,
 'remex': 1,
 'chino': 2,
 '106': 1,
 'nonprogram': 1,
 'interpret': 1,
 '105': 1,
 'luther': 1,
 'deterior': 1,
 'credit': 1,
 'r402': 1,
 'bayer': 1,
 'r408': 1,
 'controversi': 1,
 'counti': 1,
 'schur': 1,
 'unin': 1,
 '9n': 2,
 'replacd': 2,
 'avenu': 1,
 'moral': 1,
 'kaner': 1,
 'unit': 1,
 'seppanen': 1,
 'elettrotecnica': 1,
 'maer': 1,
 'music': 1,
 'telegraph': 1,
 'strike': 1,
 'swap': 1,
 'griswold': 1,
 'inverno': 1,
 'relat': 2,
 '98': 1,
 'exce': 2,
 '91': 1,
 '90': 1,
 'hole'

In [114]:
len(i.docs)

4203

In [113]:
len(prank.randomwalk(i.linkout,i.linkin))

0.000126355935807
0.00019391534015
0.000216994879784
0.00022379641448
0.000227359717881
0.000229247333327
0.000230286969538
0.000230877507717
0.000231220761536
0.00023142387018
0.000231545723788
0.000231619604292
0.000231664751965
0.000231692496299
0.000231709608412
0.000231720183267
0.00023172672046
0.000231730756662
0.000231733241572
0.000231734764291
0.000231735691065
0.000231736249811
0.00023173658232
0.00023173677666
0.000231736887374
0.000231736948091
0.000231736979413
0.000231736993857
0.000231736998937
0.000231736999098
0.000231736996907
0.000231736993807
0.000231736990567
0.000231736987565
0.000231736984958
0.000231736982785
0.000231736981023
0.000231736979621
0.000231736978523
0.000231736977673
0.000231736977021
0.000231736976524
0.000231736976148
0.000231736975865
0.000231736975653
0.000231736975494
0.000231736975376
0.000231736975289
0.000231736975224
0.000231736975175
0.00023173697514
0.000231736975114
0.000231736975095
0.00023173697508
0.00023173697507
0.000231736975062
0

4204

In [67]:
# w.index.linkout

In [103]:
from queryparser import *

In [105]:
gq=QueryCACM()
gq.relevants()

In [106]:
gq.Querycacm()

{'1': <queryparser.Query instance at 0x10b2851b8>,
 '10': <queryparser.Query instance at 0x10e2331b8>,
 '11': <queryparser.Query instance at 0x10e233e18>,
 '12': <queryparser.Query instance at 0x10e2332d8>,
 '13': <queryparser.Query instance at 0x10e233a70>,
 '14': <queryparser.Query instance at 0x10e233f38>,
 '15': <queryparser.Query instance at 0x10e233e60>,
 '16': <queryparser.Query instance at 0x10e2330e0>,
 '17': <queryparser.Query instance at 0x10e233ab8>,
 '18': <queryparser.Query instance at 0x10e233878>,
 '19': <queryparser.Query instance at 0x10e2337a0>,
 '2': <queryparser.Query instance at 0x10b285290>,
 '20': <queryparser.Query instance at 0x10e233320>,
 '21': <queryparser.Query instance at 0x10e233908>,
 '22': <queryparser.Query instance at 0x10e233f80>,
 '23': <queryparser.Query instance at 0x10e233368>,
 '24': <queryparser.Query instance at 0x10e233fc8>,
 '25': <queryparser.Query instance at 0x10e2339e0>,
 '26': <queryparser.Query instance at 0x10b3b4a28>,
 '27': <queryp

In [ ]:
prank